In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from datetime import datetime

In [28]:
data = []
page = 1
max_page = 1

In [29]:
while page <= max_page:
    res = requests.get(f'https://ryvok.ru/instrumenty/shlifmashiny/?page={page}')
    soup = BeautifulSoup(res.text, 'html')


    elements = soup.find_all('div', class_ = 'app-product-card__bottom')


    for element in elements:
        data.append({
            'title': element.find('div', class_ = 'app-product-card__title').text.strip(),
            'status': element.find('div', class_ = 'app-product-card__status').text.strip(),
            'price': element.find('div', class_ = 'app-product-card__price').text.strip().replace('\xa0', '').replace(' ₽', ''),
        })

    pagination = soup.find('div', class_ = 'ui-pagination__numbers')
    max_page = int(pagination.find_all('div', class_ = 'ui-pagination__number')[-1].text)
    page += 1
    # print(page, max_page)


In [30]:
df = pd.DataFrame(data)
df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [31]:
gc = gspread.service_account(filename='creds.json')
wks = gc.open("parsing-1").sheet1
current_df = pd.DataFrame(wks.get_all_records())

In [32]:
merged_df = pd.concat([df, current_df])
merged_df.shape
wks.update([merged_df.columns.values.tolist()] + merged_df.values.tolist())

{'spreadsheetId': '1m_ffckm9EMMKy2jBIT59e7oYWZfuDG5Fg6kDp1q_kdg',
 'updatedRange': "'Лист1'!A1:D2170",
 'updatedRows': 2170,
 'updatedColumns': 4,
 'updatedCells': 8680}